In [25]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

batch_size =32
num_classes = 10
learning_rate = 0.001
epochs = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt 


all_transforms = transforms.Compose([
                                transforms.Resize((32,32)),
                                transforms.ToTensor()
                    ]
)

training_data = datasets.FashionMNIST(
        root = "./data",
        train = True,
        download = True,
        transform = all_transforms
)

test_data = datasets.FashionMNIST(
        root="./data",
        train = False,
        download = True,
        transform = all_transforms
)

train_loader = torch.utils.data.DataLoader(dataset = training_data,
                                            batch_size = batch_size,
                                            shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_data,
                                            batch_size = batch_size,
                                            shuffle = True)


In [31]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv_layer3 = nn.Conv2d(in_channels=64,out_channels=128, kernel_size=3)
        self.conv_layer4 =nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(3200, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)

        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)

        out = out.reshape(out.size(0),-1)

        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [35]:
model = CNN(num_classes)
model = model.to(device)
# loss function
criteria = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9,weight_decay=0.005 )

total_step = len(train_loader)

In [36]:
for epoch in range(epochs):
    for data in train_loader:
        images,labels = data
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criteria(outputs,labels)

        #Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}, Loss: {:4f}]'.format(epoch+1, epochs, loss.item()))

Epoch [1/5, Loss: 0.666473]
Epoch [2/5, Loss: 0.552334]
Epoch [3/5, Loss: 0.550636]
Epoch [4/5, Loss: 0.444051]
Epoch [5/5, Loss: 0.297644]


In [34]:
# Testing model
with torch.no_grad():
    correct = 0
    total =0
    for data in test_loader:
        images = images.to(device)
        labels = images.to(device)

        outputs = model(images)

        _, predicted = torch.max(outputs.data,1)
        total+= labels.size(0)
        correct+= (predicted == labels).sum().item()

    print('Accuracy : {}%'.format(correct/total))

Accuracy : 70.0625%
